In [1]:
from keras.layers import Input, Dense, Dropout
from keras.layers import Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Flatten
import keras.layers as KL
from keras.models import Model
import keras

from training_utils import gen_classifier_dataset

import numpy as np
import json
import time
import os

Using TensorFlow backend.


# Global Config

In [2]:
IMAGE_SHAPE = (200, 200, 1)
NUM_LABELS = 5
BINS_EDGE = np.load("./data/bins_edge.npy")
NUM_CLASSES = len(BINS_EDGE) - 1  

with open('./data/classes_weight.json', 'r') as fp:
    CLASSES_WEIGHT = json.load(fp)

# 1. Model Architect
## 1.1 ResNet Block
Definition of ResNet block is adapted from https://github.com/uzh-rpg/rpg_public_dronet

In [3]:
# Before first block

img_input = Input(shape=IMAGE_SHAPE)

x0 = Conv2D(filters = 32, activation = None,
            kernel_size = (5, 5), strides=2, 
            padding='same')(img_input)

x0 = MaxPooling2D(pool_size=(3, 3), strides=[2,2])(x0)

# x0 is the output of the first three layers
#    and the input of the first residual block

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def build_residual_block(block_input, filt_size):
    """
    #Define residual block.
    #
    # Arguments
    #   block_input: The input of the block
    #   filt_size: Block filter size (convolutional 2D kernel)
    #   
    # Returns
    #   block_output: The output of the block
    """
  
    block_output = BatchNormalization()(block_input)
    
    block_output = Activation('relu')(block_output)

    block_output = Conv2D(filters = filt_size, strides=2,
                          kernel_size = 3,
                          activation = 'linear', padding='same',
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(1e-4))(block_output)

    block_output = BatchNormalization()(block_output)

    block_output = Activation('relu')(block_output)
    
    block_output = Conv2D(filters = filt_size, strides=1,
                          kernel_size = 3,
                          activation = 'linear', padding='same',
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(1e-4))(block_output)
    
    return block_output

Function to build the block with the convolutional shortcut:

In [5]:
def build_full_block(block_input, filt_size):
    """
    # Define full block.
    #
    # Arguments
    #   block_input: The input of the full block
    #   filt_size: Filter size (conv2D kernel)
    #   
    # Returns
    #   block_output: The output of the block
    """
  
    # First residual block
    x_main = build_residual_block(block_input, filt_size)
    # x_main = "output of the main path"

    # First convolutional shortcut
    x_short = Conv2D(filters = filt_size,  strides = 2,
                     kernel_size = 1,
                     activation = None, padding = 'same')(block_input)
    # x_short = "output of the shortcut"

    # Add the outputs of the residual block and the convolutional shortcut
    block_output = keras.layers.add([x_main, x_short])

    return block_output

Build the three full blocks:

In [6]:
x1 = build_full_block(x0, filt_size = 32 )
x2 = build_full_block(x1, filt_size = 64 )
x3 = build_full_block(x2, filt_size = 128)

Define the flatten layer, apply one time the ReLU activation function and use a dropout of 0.5.

Only two Dense layer as output. NVIDIA paper uses 3 Dense layer. 

In [7]:
x4 = Flatten()(x3)
x4 = Activation('relu')(x4) # Apply ReLU to every output from the Flatten layer
x4 = Dropout(0.5)(x4) # Conserve only 0.5 of the results

# Steering channel (output)
steer = Dense(800, activation="relu")(x4)
steer = Dropout(0.5)(steer) # Conserve only 0.5 of the results
steer = Dense(NUM_CLASSES,activation = "softmax")(steer) 
# Activation:
# --> sigmoid = binary classification
# --> softmax = categorical classification 


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 1.4 Create Model
Finally put everything in the model:

In [8]:
# Define steering-collision model
model = Model(inputs=[img_input], outputs=[steer])

In [9]:
print (model.input)
print (model.output)
print (model.summary())

Tensor("input_1:0", shape=(?, 200, 200, 1), dtype=float32)
Tensor("dense_2/Softmax:0", shape=(?, 114), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 49, 49, 32)   128         max_pooling2d_1[0][0]    

#### Compile the model:

In [10]:
model.compile(keras.optimizers.Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 2. Training

In [11]:
train_path_X = "./data/CH2_training_X.npy"
train_path_y = "./data/CH2_training_y.npy"
val_path_X = "./data/CH2_training_X.npy"
val_path_y = "./data/CH2_training_y.npy"

In [12]:
def load_dataset(path_to_dataset_X, path_to_dataset_y):
    """
    Load train/validation set from .npy file
    Input:
        path_to_dataset (str)
    Output:
        X (np.ndarray): shape (num_samples, image_height, image_width, 1) 
        y (list): each element is a np.ndarray, shape (num_samples, num_classes)
    """
    X = np.load(path_to_dataset_X)
    y_tensor = np.load(path_to_dataset_y)
    y = [y_tensor[i, :, :]for i in range(NUM_LABELS)]
    return X, y


gen_param = {'num_classes': NUM_CLASSES, 
             'num_labels': NUM_LABELS, 
             'bins_edge': BINS_EDGE, 
             'image_shape': IMAGE_SHAPE, 
             'num_samples': None, 
             'data_root_dir': "./data/training_data/", # path to folder contained images
             'flip_prob': 0.5}

if os.path.isfile(train_path_X):
    print("Load dataset")
    X_train, y_train = load_dataset(train_path_X, train_path_y)
else:
    print("Generate dataset")
    X_train, y_train = gen_classifier_dataset("./data/CH2_training.csv", **gen_param)
    # save data file for future use
    np.save('./data/CH2_training_X.npy', X_train)
    np.save('./data/CH2_training_y.npy', y_train)

if os.path.isfile(val_path_X):
    X_val, y_val = load_dataset(val_path_X, val_path_y)
else:
    X_val, y_val = gen_classifier_dataset("./data/CH2_training.csv", **gen_param)
    np.save('./data/CH2_validation_X.npy', X_val)
    np.save('./data/CH2_validation_y.npy', y_val)

Load dataset


MemoryError: 

#### Adapting dataset to see-1-predict-1 architecture
The data is preprared for see-1-predict-5 architecture, so we have 5 labels for each. 
We will just take the first one:

In [ ]:
y_train = y_train[0]
y_val = y_val[0]

In [ ]:
# Integer conversion of the keys from char to integers
classes_weight = {}
for k in CLASSES_WEIGHT.keys():
    classes_weight[int(k)] = CLASSES_WEIGHT[k]

In [ ]:
batch_size = 200
max_epochs = 1000

time_str = time.strftime("%Y_%m_%d_%H_%M")
log_dir = './logs/' + time_str

# See: https://keras.io/callbacks/#tensorboard
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                          batch_size=batch_size, 
                                          update_freq='epoch',
                                          histogram_freq = 1,
                                          write_images = True) 

early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=50,
                                              restore_best_weights=True)

history = model.fit(x=X_train,
                y=y_train,
                epochs=max_epochs,
                validation_data=(X_val, y_val),
                callbacks=[tb_callback, early_stop_cb],
                class_weight=classes_weight,
                initial_epoch=0,
                verbose = 2,
                workers = 10,
                shuffle=True,
                batch_size=batch_size)


# 3. Plot Training Result

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# 4. Save the model:

In [ ]:
# serialize model to HDF5
model.save(log_dir + "/see_1_predict_1_%s.h5" % time_str)